# Building optimization models from pandas dataframes

`gurobipy-pandas`: The missing link between `pandas` and `gurobipy`!

Key points:

- Pandas style is to avoid writing explicit loops and instead do high level declarative operations. This library enables that style for writing optimization models.
- This is not the only way to write Gurobi models using pandas data.
- This library does not guarantee super-fast code. Rather, following the style enabled by this library should avoid some of the major performance bugs we've seen in user code that mixes gurobipy and pandas.
- You should prepare your data in advance, to match your mathematical model. There are nice crossovers between pandas structures and the parts of an optimization model, which you can exploit to produce clear code without unnecessary overhead.

Todo:

- Try to disambiguate symbols (use more of the alphabet to avoid redefining, x, y, z, c, etc)
- Review the order of expression building examples (should build up to the difficult result)

In this session:

- Overview of `gurobipy-pandas` design
- Basic usage mechanics
    - Creating variables
    - Building expressions using series of variables and data
    - Creating constraints
    - Retrieving solutions
- A complete modelling example

# Design Principles

- Optimization models define all data, variables, and constraints over indexes

$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{ij} x_{ij} \\
\mbox{s.t.} \quad & \sum_{i \in J_i} w_{ij} x_{ij} \le c_{i} & \forall i \in I \\
                  & x_{ij} \in \lbrace 0, 1 \rbrace & \forall i \in I, j \in J_i \\
\end{alignat}
$$

- The mathematical indices provide a clear way to structure data in code

In [1]:
# Just some random data to show.
import pandas as pd
import numpy as np
product_data = pd.DataFrame(
    index=pd.MultiIndex.from_tuples([(1, 2), (1, 1), (2, 3), (2, 1), (3, 1)]),
    columns=["cost_per_unit", "amount_available"],
    data=np.random.random((5, 2)).round(2)
)

# Design Principles

- Pandas DataFrames and Series *also* define data over indexes

In [2]:
product_data

cost_per_unit  amount_available
1 2           0.97              0.39
  1           0.70              0.78
2 3           0.18              0.12
  1           0.39              0.16
3 1           0.36              0.30

- We just need a way to define variables and build constraints over those indexes

# Missing Link?

`gurobipy-pandas` provides:

- Methods to create pandas-indexed series of variables
- Methods to build constraints from expressions
- Accessors to extract solutions as pandas structures

`pandas` provides:

- Existing algebraic/split-apply-combine logic (well known transforms)

- With pandas, we are already set up to define all our data over various indexes, aligned with the mathematical model.
- What we are missing:
    - Easy way to define/construct variables over the same indexes using pandas structures
    - Methods to transform data and variables into constraint expressions (actually, we get this for free)
    - Easy way to take the resulting expressions and build constraints from them

# Installation

```
pip install gurobipy-pandas
```

Then, add one more import to your usual arsenal:

In [3]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

import gurobipy_pandas as gppd

# Handy trick for live coding
gppd.set_interactive()

# Silence please
gp.setParam('OutputFlag', 0)

# Usage

- `gurobipy` objects are still the entry point for:
    - creating models
    - starting optimization
    - constants, status codes, etc
- `gurobip-pandas` provides accessors and functions to:
    - create sets of variables based on indexes
    - create constraints based on aligned series
    - extract solutions as series

# Creating Models

The usual way; entry-point doesn't change

In [4]:
model = gp.Model()

# Creating Variables

- Using free functions

- Free function: returns a series of variables (not part of a dataframe)
- Variables live in the namespace, not as columns
- Returned series index matches the index of the dataframe

In [5]:
data = pd.DataFrame(
    {
        "i": [0, 0, 1, 2, 2],
        "j": [1, 2, 0, 0, 1],
        "profit": [0.3, 1.2, 0.7, 0.9, 1.2],
        "weight": [1.3, 1.7, 1.4, 1.1, 0.9],
    }
).set_index(["i", "j"])

In [6]:
data

profit  weight
i j                
0 1     0.3     1.3
  2     1.2     1.7
1 0     0.7     1.4
2 0     0.9     1.1
  1     1.2     0.9

In [7]:
x = gppd.add_vars(model, data, name="x", obj="profit")
x

i  j
0  1    <gurobi.Var x[0,1]>
   2    <gurobi.Var x[0,2]>
1  0    <gurobi.Var x[1,0]>
2  0    <gurobi.Var x[2,0]>
   1    <gurobi.Var x[2,1]>
Name: x, dtype: object

# Creating Variables

- Using dataframe accessors

- Dataframe accessor: returns a new dataframe with an appended column
- Use this approach to allow method chaining
- Variables live as columns in a dataframe, along with other data/objects sharing that index

In [8]:
data

profit  weight
i j                
0 1     0.3     1.3
  2     1.2     1.7
1 0     0.7     1.4
2 0     0.9     1.1
  1     1.2     0.9

In [9]:
variables = (
    data
    .gppd.add_vars(model, name="y", ub=100.0)
    .gppd.add_vars(model, name="z", obj="profit")
)
variables

profit  weight                    y                    z
i j                                                          
0 1     0.3     1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>
  2     1.2     1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>
1 0     0.7     1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>
2 0     0.9     1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>
  1     1.2     0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>

That's about all there is to it; one variable is created per index entry

# Creating Expressions

- Pandas handles this for us
- We always leverage pandas-native functions
- Common operations in mathprog:
    - summation
    - arithmetic operations
    - groupby (split-apply-combine) operations
- Let's explore some common mathematical expressions (note: showing these as series)

## Single indexes

Consider an index $i \in I$, some variables $x_i$, and some data $c_i$

In [10]:
i = pd.RangeIndex(5, name="i")
i

RangeIndex(start=0, stop=5, step=1, name='i')

In [11]:
x = gppd.add_vars(model, i, name="x")
x

i
0    <gurobi.Var x[0]>
1    <gurobi.Var x[1]>
2    <gurobi.Var x[2]>
3    <gurobi.Var x[3]>
4    <gurobi.Var x[4]>
Name: x, dtype: object

In [12]:
c = pd.Series(index=i, name="c", data=np.arange(1, 6))
c

i
0    1
1    2
2    3
3    4
4    5
Name: c, dtype: int64

## Arithmetic with scalars

$$
2 x_i + 5
$$

In [13]:
2*x + 5

i
0    5.0 + 2.0 x[0]
1    5.0 + 2.0 x[1]
2    5.0 + 2.0 x[2]
3    5.0 + 2.0 x[3]
4    5.0 + 2.0 x[4]
Name: x, dtype: object

## Summation

$$
\sum_i x_i
$$

In [14]:
x.sum()

<gurobi.LinExpr: x[0] + x[1] + x[2] + x[3] + x[4]>

## Arithmetic with Series

$$
c_i x_i
$$

In [15]:
c * x

i
0        x[0]
1    2.0 x[1]
2    3.0 x[2]
3    4.0 x[3]
4    5.0 x[4]
dtype: object

## Summing the result

$$
\sum_i c_i x_i
$$

In [16]:
(c * x).sum()

<gurobi.LinExpr: x[0] + 2.0 x[1] + 3.0 x[2] + 4.0 x[3] + 5.0 x[4]>

## Comfortable/Familiar?

Hopefully!

Any operation you would do with data in pandas, you can do in the same way with data & variables.

Let's look at some multi-index cases.

For each $i$, sum over all corresponding $j$:

$$
\sum_{i \in I} x_{ij} \quad \forall j \in J
$$

In [17]:
x = gppd.add_vars(model, data, name="x")
x

i  j
0  1    <gurobi.Var x[0,1]>
   2    <gurobi.Var x[0,2]>
1  0    <gurobi.Var x[1,0]>
2  0    <gurobi.Var x[2,0]>
   1    <gurobi.Var x[2,1]>
Name: x, dtype: object

In [18]:
x.groupby("i").sum()

i
0        x[0,1] + x[0,2]
1    <gurobi.Var x[1,0]>
2        x[2,0] + x[2,1]
Name: x, dtype: object

Correspondence:

- index specified in `groupby` matches `\forall`
- `.sum()` reduces over the remaining index levels

Align data on partial indexes:

$$
c_i x_{ij}
$$

In [19]:
c = pd.Series(index=pd.RangeIndex(3, name='i'), data=[1, 2, 3], name="c")
c

i
0    1
1    2
2    3
Name: c, dtype: int64

In [20]:
c * x

i  j
0  1        x[0,1]
   2        x[0,2]
1  0    2.0 x[1,0]
2  0    3.0 x[2,0]
   1    3.0 x[2,1]
dtype: object

Align the other way (by *names*):

$$
b_j x_{ij}
$$

In [21]:
b = pd.Series(index=pd.RangeIndex(3, name='j'), data=[1, 2, 3], name="b")
b

j
0    1
1    2
2    3
Name: b, dtype: int64

In [22]:
b * x

i  j
0  1    2.0 x[0,1]
   2    3.0 x[0,2]
1  0        x[1,0]
2  0        x[2,0]
   1    2.0 x[2,1]
dtype: object

Note this alignment is all performed by pandas, according to pandas rules

Slightly more complex:

$$
\sum_j b_j x_{ij} \quad \forall i \in I
$$

Here we are just taking the series `b * x` and applying the same groupby-aggregate operation as before:

In [23]:
(b * x).groupby("i").sum()

i
0    2.0 x[0,1] + 3.0 x[0,2]
1                     x[1,0]
2        x[2,0] + 2.0 x[2,1]
dtype: object

Note that we get a new series, with index $i$. This will be important shortly...

# Creating Constraints

- Using free functions


$$
\sum_j b_j x_{ij} \le 1 \quad \forall i \in I
$$

In [24]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    1.0,
    name="c1",
)

i
0    <gurobi.Constr c1[0]>
1    <gurobi.Constr c1[1]>
2    <gurobi.Constr c1[2]>
Name: c1, dtype: object

## Index alignment / Missing data

- To use series on both sides, indexes must align

In [25]:
c = pd.Series(index=pd.RangeIndex(3, name="i"), data=[1, 2, 3])
c

i
0    1
1    2
2    3
dtype: int64

In [26]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    c,
    name="c1",
)

i
0    <gurobi.Constr c1[0]>
1    <gurobi.Constr c1[1]>
2    <gurobi.Constr c1[2]>
Name: c1, dtype: object

- Unaligned data, or missing data, is an error

In [27]:
gppd.add_constrs(  
    model,
    (b * x).groupby("i").sum(),
    GRB.LESS_EQUAL,
    pd.Series(index=pd.RangeIndex(4, name="i"), data=[1, 2, 3, 4]),
    name="c1",
)

KeyError: 'series must be aligned'

# Creating Constraints

- Using dataframe accessors

In [28]:
vars_and_constrs = variables.gppd.add_constrs(  
    model, "y + z <= 1", name="c1"
)
vars_and_constrs

profit  weight                    y                    z  \
i j                                                             
0 1     0.3     1.3  <gurobi.Var y[0,1]>  <gurobi.Var z[0,1]>   
  2     1.2     1.7  <gurobi.Var y[0,2]>  <gurobi.Var z[0,2]>   
1 0     0.7     1.4  <gurobi.Var y[1,0]>  <gurobi.Var z[1,0]>   
2 0     0.9     1.1  <gurobi.Var y[2,0]>  <gurobi.Var z[2,0]>   
  1     1.2     0.9  <gurobi.Var y[2,1]>  <gurobi.Var z[2,1]>   

                          c1  
i j                           
0 1  <gurobi.Constr c1[0,1]>  
  2  <gurobi.Constr c1[0,2]>  
1 0  <gurobi.Constr c1[1,0]>  
2 0  <gurobi.Constr c1[2,0]>  
  1  <gurobi.Constr c1[2,1]>

# Setting Objectives

Generally one objective (not an index-wise operation)

So we use `gurobipy` methods here

In [29]:
(x * data["profit"]).sum()

<gurobi.LinExpr: 0.3 x[0,1] + 1.2 x[0,2] + 0.7 x[1,0] + 0.9 x[2,0] + 1.2 x[2,1]>

In [30]:
model.setObjective((x * data["profit"]).sum(), sense=GRB.MAXIMIZE)

Can also use `obj="col"`

# Extracting Solutions

(Solve the model first)

In [31]:
model.optimize()
x.gppd.X  # Series accessor

i  j
0  1    0.000000
   2    0.333333
1  0    1.000000
2  0    1.000000
   1    0.000000
Name: x, dtype: float64

# Other Series accessor functionality

In [32]:
vars_and_constrs['c1']

i  j
0  1    <gurobi.Constr c1[0,1]>
   2    <gurobi.Constr c1[0,2]>
1  0    <gurobi.Constr c1[1,0]>
2  0    <gurobi.Constr c1[2,0]>
   1    <gurobi.Constr c1[2,1]>
Name: c1, dtype: object

In [33]:
vars_and_constrs['c1'].gppd.Slack

i  j
0  1    1.0
   2    1.0
1  0    1.0
2  0    1.0
   1    1.0
Name: c1, dtype: float64

In [34]:
expr = 2.0 * x + variables['y'] + 0.5
expr

i  j
0  1    0.5 + 2.0 x[0,1] + y[0,1]
   2    0.5 + 2.0 x[0,2] + y[0,2]
1  0    0.5 + 2.0 x[1,0] + y[1,0]
2  0    0.5 + 2.0 x[2,0] + y[2,0]
   1    0.5 + 2.0 x[2,1] + y[2,1]
dtype: object

In [35]:
expr.gppd.get_value()

i  j
0  1    0.500000
   2    1.166667
1  0    2.500000
2  0    2.500000
   1    0.500000
dtype: float64

# Example

[Project-Team Allocation](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples/projects.html). Focus on usage in practice. Key points:

- *Before* taking any modelling steps: prepare your data properly
    - Clearly define your model indexes, and align your dataframes to these indexes
    - Keep data reading & cleaning separate from model building
- Solutions are read back as numeric pandas data
    - Plug directly into the rest of pydata ecosystem for post-processing
    - Work pandonically with the results

In [36]:
projects = pd.read_csv("docs/source/examples/data/projects.csv", index_col="project")
teams = pd.read_csv("docs/source/examples/data/teams.csv", index_col="team")
allowed_pairs = (
    pd.merge(
        projects.reset_index(),
        teams.reset_index(),
        how='cross',
    )
    .query("difficulty <= skill")
    .set_index(["project", "team"])
)
projects = projects.drop(columns=["difficulty", "value"])
teams = teams.drop(columns=["skill"])
project_values = allowed_pairs[["value"]].rename(columns={"value": "profit"})

# The Model

- Given a set of projects $i \in I$ and teams $j \in J$.
- Project $i$ requires $w_i$ resources to complete, and each team $j$ has capacity $c_j$.
- If team $j$ completes project $i$, we get profit $p_{ij}$.

$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{ij} x_{ij} \\
\mbox{s.t.} \quad & \sum_{i \in I} w_{i} x_{ij} \le c_{j} & \forall j \in J \\
                  & \sum_{j \in J} x_{ij} \le 1 & \forall i \in I \\
                  & x_{ij} \in \lbrace 0, 1 \rbrace & \forall i \in I, j \in J \\
\end{alignat}
$$

# The Data



In [37]:
projects.head(3)

,resource
project,
0,1.1
1,1.4
2,1.2


In [38]:
teams

,capacity
team,
0,2.4
1,1.8
2,1.1
3,1.9
4,1.4


## Sparsity

- Note that the model is not defined over all $(i, j)$ pairs
- (Not all teams can complete all projects)
- There are 80 < 150 combinations (sparse!)
- This is **well-prepared** data

In [39]:
project_values

profit
project team        
0       4        0.4
1       4        1.3
2       0        1.7
        1        1.7
        2        1.7
...              ...
28      2        1.0
        3        1.0
        4        1.0
29      3        1.3
        4        1.3

[80 rows x 1 columns]

## Clean data

- There are no missing values
- All indexes ($i$, $j$, $ij$) are correctly represented
- Index alignment is correct (every project represented in project_values has an entry in projets, etc)

## Define variables and objective

$$
\begin{alignat}{2}
\max \quad        & \sum_{i \in I} \sum_{j \in J} p_{ij} x_{ij} \\
\mbox{s.t.} \quad & x_{ij} \in \lbrace 0, 1 \rbrace & \forall i \in I, j \in J \\
\end{alignat}
$$

In [40]:
model = gp.Model()
model.ModelSense = GRB.MAXIMIZE
x = gppd.add_vars(model, project_values, vtype=GRB.BINARY, obj="profit", name="x")
x

project  team
0        4        <gurobi.Var x[0,4]>
1        4        <gurobi.Var x[1,4]>
2        0        <gurobi.Var x[2,0]>
         1        <gurobi.Var x[2,1]>
         2        <gurobi.Var x[2,2]>
                         ...         
28       2       <gurobi.Var x[28,2]>
         3       <gurobi.Var x[28,3]>
         4       <gurobi.Var x[28,4]>
29       3       <gurobi.Var x[29,3]>
         4       <gurobi.Var x[29,4]>
Name: x, Length: 80, dtype: object

## Capacity constraint

$$
\sum_{i \in I} w_{i} x_{ij} \le c_{j} \quad \forall j \in J
$$

In [41]:
capacity_constraints = gppd.add_constrs(
    model,
    (
        (projects["resource"] * x)
        .groupby("team").sum()
    ),
    GRB.LESS_EQUAL,
    teams["capacity"],
    name='capacity',
)
capacity_constraints.head()

team
0    <gurobi.Constr capacity[0]>
1    <gurobi.Constr capacity[1]>
2    <gurobi.Constr capacity[2]>
3    <gurobi.Constr capacity[3]>
4    <gurobi.Constr capacity[4]>
Name: capacity, dtype: object

## Allocate once

$$
\sum_{j \in J} x_{ij} \le 1 \quad \forall i \in I
$$

In [42]:
allocate_once = gppd.add_constrs(
    model, x.groupby('project').sum(),
    GRB.LESS_EQUAL, 1.0, name="allocate_once",
)
allocate_once.head()

project
0    <gurobi.Constr allocate_once[0]>
1    <gurobi.Constr allocate_once[1]>
2    <gurobi.Constr allocate_once[2]>
3    <gurobi.Constr allocate_once[3]>
4    <gurobi.Constr allocate_once[4]>
Name: allocate_once, dtype: object

## Solutions

- Solve the model
- Get back solution values as a series on our original index

In [43]:
model.optimize()
x.gppd.X.head()

project  team
0        4       0.0
1        4       0.0
2        0      -0.0
         1       1.0
         2       0.0
Name: x, dtype: float64

In [44]:
(
    x.gppd.X.to_frame()
    .query("x >= 0.9").reset_index()
    .groupby("team").agg({"project": list})
)

,project
team,
0,"[4, 5]"
1,[2]
2,[11]
3,"[6, 29]"
4,"[14, 15, 26]"


- Apply a little pandas magic
- This is just to emphasise that the results are pandas-y
- Extract them, transform them, write them somewhere

# More examples are available

This was an abriged version of the project-team allocation example

https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/examples.html

# Performance

[Dedicated page in the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/performance.html). Key points:

- `gurobipy-pandas` won't magically make your model building code fast
- The API provides some structure to take *well-organized and prepared data* and help you build *clearly defined models* in the pandas style

# Final thoughts

- [Read the docs](https://gurobi-optimization-gurobipy-pandas.readthedocs-hosted.com/en/latest/index.html)
    - Check the examples for clean patterns to emulate
- [Check the repo](https://github.com/Gurobi/gurobipy-pandas)
    - Bugs? Feature request? Open an issue
- [Discuss!](https://support.gurobi.com/hc/en-us/community/topics/10373864542609-GitHub-Projects%3E)
    - For usage questions

# Thanks!

Questions?